In [1]:
import numpy as np
from math import pi, tau

from datetime import datetime, timezone
from tzwhere import tzwhere
import pytz
import suncalc

from colorsys import hls_to_rgb
from PIL import Image
# import drawSvg as draw

In [2]:
# azimuth, altitude to color
def get_color(azimuth, altitude, sunrise_jump=0.2, hue_shift=0.0, as_hex=True):
    # ranges from 0 (no jump) to 1 (day is all white, night all black)
    assert 0 <= sunrise_jump <= 1, "sunrise_jump must be between 0 and 1 inclusive"

    # can be any float, but values outside the range [0, 1) are redundant
    # shifts all hues in the r -> g -> b -> r direction
    assert isinstance(hue_shift, float), "hue_shift must be a float"

    # yes, this could be simplified, and no, don't try to do it please.
    altitude_scaled = (altitude / pi) * 2  # range [-1, 1]
    altitude_scaled *= 1 - sunrise_jump
    altitude_scaled += sunrise_jump * (1 if altitude >= 0 else -1)  # range [-1, 1]
    lightness = altitude_scaled / 2 + 0.5  # range [0, 1]

    hue = ((azimuth / tau) + 0.5 + hue_shift) % 1  # range [0, 1]

    r, g, b = hls_to_rgb(hue, lightness, 1)
    r = round(255 * r)
    g = round(255 * g)
    b = round(255 * b)
    return "{0:02x}{1:02x}{2:02x}".format(r, g, b).upper() if as_hex else r, g, b

In [3]:
# array of UTC timestamps as naive datetime objects
def base_date_arr(year: int):
    start_time = np.datetime64(str(year))
    end_time = np.datetime64(str(year + 1))

    arr_dt_1d = np.arange(start_time, end_time, dtype='datetime64[m]').astype(datetime)  # 1D array
    arr_utc_2d = arr_dt_1d.reshape(-1, 1440).transpose()  # 2D array with days as columns, time flowing top to bottom

    # flip array vertically since SVG is filled from bottom left corner
    return arr_utc_2d

In [4]:
# array of naive timestamps to array of time zone aware timestamps
def to_utc(arr, lon, lat, use_dst: bool):
    tz_str = tzwhere.tzwhere().tzNameAt(lon, lat)
    tz = pytz.timezone(tz_str)

    def one_dt_utc(dt):
        return (dt - offset).replace(tzinfo=timezone.utc)

    def one_dt_localized(dt):
        return tz.localize(dt).astimezone(pytz.utc)

    if use_dst:
        return np.vectorize(one_dt_localized)(arr)
    else:
        offset = tz.utcoffset(arr[0, 0], is_dst=False)
        return np.vectorize(one_dt_utc)(arr)

In [5]:
# array of timestamps to sun positions, then to [r, g, b]]
def pos_png(arr_dt, lon, lat, sunrise_jump=0.0, hue_shift=0.0):
    # get shape of input array
    shape_old = arr_dt.shape
    # create empty array with old shape and depth 3
    rgb_arr = np.empty((shape_old[0], shape_old[1], 3), dtype=np.uint8)
    
    r_lst = []
    g_lst = []
    b_lst = []

    for row_idx in range(shape_old[0]):
        for col_idx in range(shape_old[1]):
            azi_alt = suncalc.get_position(arr_dt[row_idx, col_idx], lon, lat)
            r, g, b = get_color(azi_alt['azimuth'], azi_alt['altitude'],
                                sunrise_jump=sunrise_jump, hue_shift=hue_shift, as_hex=False)
            rgb_arr[row_idx, col_idx, 0] = r
            rgb_arr[row_idx, col_idx, 1] = g
            rgb_arr[row_idx, col_idx, 2] = b
            r_lst.append(r)
            g_lst.append(g)
            b_lst.append(b)

    return rgb_arr, r_lst, g_lst, b_lst

In [6]:
# generate PNG using pixel data
def gen_png(rgb_arr, width, height, file_name):
    if not '.png' in file_name: file_name = file_name + '.png'
    Image.fromarray(rgb_arr, mode="RGB") \
        .resize((rgb_arr.shape[1], height), Image.BOX) \
        .resize((width, height), Image.NEAREST) \
        .save(file_name)

In [7]:
year = 2021

img_title = 'Melbourne'
lon = 144.9631
lat = -37.8136

width = 1920
height = 1080

print('Building array of datetime objects...')
bef = datetime.now()
arr_dt = base_date_arr(year)  # naive timestamps as datetime.datetime
af = bef = datetime.now() - bef
print(f'Took {af}')
print('Converting dates to UTC...')
bef = datetime.now()
arr_utc = to_utc(arr_dt, lat, lon, use_dst=True)
af = bef = datetime.now() - bef
print(f'Took {af}')
print('Calculating sun location and corresponding colors...')
bef = datetime.now()
pixels, r, g, b = pos_png(arr_utc, lon, lat, sunrise_jump=0.3, hue_shift=0.0)
af = bef = datetime.now() - bef
print(f'Took {af}')

print('Generating image...')
bef = datetime.now()
gen_png(pixels, width, height, img_title)
af = bef = datetime.now() - bef
print(f'Took {af}')

Building array of datetime objects...
Took 0:00:00.045132
Converting dates to UTC...


/home/anoush/.local/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Took 0:00:13.469239
Calculating sun location and corresponding colors...
Took 0:00:24.708404
Generating image...
Took 0:00:00.269312


In [8]:
# azimuth, altitude to color
# def get_color_vec(azimuth, altitude, sunrise_jump=0.2, hue_shift=0.0):
#     # ranges from 0 (no jump) to 1 (day is all white, night all black)
#     assert 0 <= sunrise_jump <= 1, "sunrise_jump must be between 0 and 1 inclusive"

#     # can be any float, but values outside the range [0, 1) are redundant
#     # shifts all hues in the r -> g -> b -> r direction
#     assert isinstance(hue_shift, float), "hue_shift must be a float"

#     # yes, this could be simplified, and no, don't try to do it please.
#     altitude_scaled = (altitude / pi) * 2  # range [-1, 1]
#     altitude_scaled *= 1 - sunrise_jump
#     # altitude_scaled += sunrise_jump * (1 if altitude >= 0 else -1)  # range [-1, 1]
#     idx_alt_pos = altitude_scaled >= 0
#     idx_alt_neg = altitude_scaled < 0
#     altitude_scaled[idx_alt_pos] += sunrise_jump * 1
#     altitude_scaled[idx_alt_pos] += sunrise_jump * -1
#     lightness = altitude_scaled / 2 + 0.5  # range [0, 1]

#     hue = ((azimuth / tau) + 0.5 + hue_shift) % 1  # range [0, 1]
    
#     return hue, lightness

#     r, g, b = hls_to_rgb(hue, lightness, 1)
#     r = round(255 * r)
#     g = round(255 * g)
#     b = round(255 * b)
#     return "{0:02x}{1:02x}{2:02x}".format(r, g, b).upper() if as_hex else r, g, b

In [9]:
# array of UTC timestamps as naive datetime objects
# def base_date_arr_2(year: int):
#     start_time = np.datetime64(str(year))
#     end_time = np.datetime64(str(year + 1))

#     arr_dt_1d = np.arange(start_time, end_time, dtype='datetime64[m]').astype(datetime)  # 1D array
#     # arr_utc_2d = arr_dt_1d.reshape(-1, 1440).transpose()  # 2D array with days as columns, time flowing top to bottom

#     return arr_dt_1d

In [10]:
# arr_dt = base_date_arr_2(year)
# arr_utc = to_utc(arr_dt, lat, lon, use_dst=True)

In [11]:
# sc = suncalc.get_position(arr_utc, lon, lat) # fast

In [12]:
# h, l = get_color_vec(sc['azimuth'], sc['altitude'], sunrise_jump=0.2, hue_shift=0.0) # fast

In [13]:
# r_arr = np.array(['R','R','R','R','R'])
# g_arr = np.array(['G','G','G','G','G'])
# b_arr = np.array(['B','B','B','B','B'])

# arr_tuple = (r_arr, g_arr, b_arr)
# np.vstack(arr_tuple).reshape((-1,), order='F')

# def stack_hsv(h, s, v):
#     return np.vstack((h, s, v)).reshape((-1,), order='F') # then reshape into correct size

In [14]:
# start with h, l, s
# convert from hsl to hsv
# saturation = np.ones(h.shape)

# v = l + saturation * np.minimum(l, 1-l)

# s = np.zeros(v.shape)

# idx_nonzero = v != 0

# s[idx_nonzero] = 2 * (1 - (l / v[idx_nonzero]))

# h = h

In [15]:
# out = stack_hsv(h, s, v)

In [16]:
# colors_arr = out.reshape((1440,-1))

In [17]:
# def gen_png_2(rgb_arr, width, height, file_name):
#     if not '.png' in file_name: file_name = file_name + '.png'
#     Image.fromarray(rgb_arr, mode="HSV") \
#         .convert('RGB') \
#         .resize((rgb_arr.shape[1], height), Image.BOX) \
#         .resize((width, height), Image.NEAREST) \
#         .save(file_name)

In [18]:
# gen_png_2(colors_arr, width, height, 'test.png')

In [19]:
# r, g, b = hls_to_rgb(hue, lightness, saturation)

In [20]:
# t = np.vectorize(hls_to_rgb)

In [21]:
# h, l = get_color_vec(sc['azimuth'], sc['altitude'], sunrise_jump=0.2, hue_shift=0.0) # fast
# s = np.ones(h.shape)

In [22]:
# r, g, b = t(h, l, s)

In [23]:
# def stack_rgb(r, g, b):
#     return np.vstack((r, g, b)).reshape((-1,), order='F').reshape((1440,-1))

In [24]:
# out = stack_rgb(r, g, b)

In [25]:
# gen_png(out, width, height, 'new.png')

In [26]:
# pixels

In [27]:
# rr = np.array(r)
# gg = np.array(g)
# bb = np.array(b)

In [28]:
# gg